In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import glob
import sys, os
import datetime
import WetBulb

In [2]:
dataDir = '/dartfs-hpc/rc/lab/C/CMIG/ERA5'
year = 1980

In [11]:
tw = xr.open_dataset('%s/daily/tw_min_1980.nc'%(dataDir))

In [5]:
print('opening datasets for %d'%year)
mx2t = xr.open_dataset('%s/hourly/mx2t_hourly_%d.nc'%(dataDir, year))
# mx2t['mx2t'] -= 273.15
# mx2t.load()

opening datasets for 1980


In [4]:
sp = xr.open_dataset('%s/hourly/sp_hourly_%d.nc'%(dataDir, year))
sp.load()

<xarray.Dataset>
Dimensions:    (latitude: 721, longitude: 1440, time: 8784)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.25 359.5 359.75
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 1980-01-01 ... 1980-12-31T23:00:00
Data variables:
    sp         (time, latitude, longitude) float32 103057.195 ... 69420.875
Attributes:
    Conventions:  CF-1.6
    history:      2021-01-13 18:59:20 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

In [5]:
dp2t = xr.open_dataset('%s/hourly/dp_hourly_%d.nc'%(dataDir, year)) # UPDATE THIS!!
dp2t['d2m'] -= 273.15
dp2t.load()

<xarray.Dataset>
Dimensions:    (latitude: 721, longitude: 1440, time: 8784)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.25 359.5 359.75
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 1980-01-01 ... 1980-12-31T23:00:00
Data variables:
    d2m        (time, latitude, longitude) float32 -27.997147 ... -30.115875
Attributes:
    Conventions:  CF-1.6
    history:      2021-01-13 20:44:46 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

In [6]:
def spec_humidity(dp, sp):
    """Calculates SH automatically from the dewpt. Returns in g/kg"""
    # Declaring constants
    e0 = 6.113 # saturation vapor pressure in hPa
    # e0 and Pressure have to be in same units
    c_water = 5423 # L/R for water in Kelvin
    T0 = 273.15 # Kelvin

    # saturation vapor not required, uncomment to calculate it (units in hPa becuase of e0)
    #sat_vapor = self.e0 * np.exp((self.c * (self.temp -self.T0))/(self.temp * self.T0)) 

    #calculating specific humidity, q directly from dew point temperature
    #using equation 4.24, Pg 96 Practical Meteorolgy (Roland Stull)
    q = (622 * e0 * np.exp(c_water * (dp - T0)/(dp * T0)))/sp # g/kg 
    # 622 is the ratio of Rd/Rv in g/kg
    return q

In [7]:
dask_mx2t = mx2t.mx2t.chunk({'latitude':100, 'longitude':100, 'time':500})
dask_sp = sp.sp.chunk({'latitude':100, 'longitude':100, 'time':500})
dask_dp2t = dp2t.d2m.chunk({'latitude':100, 'longitude':100, 'time':500})

In [8]:
q=spec_humidity(dask_dp2t+273.15, dask_sp/100)

In [9]:
%%time
q.compute()

CPU times: user 4min 29s, sys: 12min 8s, total: 16min 37s
Wall time: 1min 14s


<xarray.DataArray (time: 8784, latitude: 721, longitude: 1440)>
array([[[0.3821875 , 0.3821875 , 0.3821875 , ..., 0.3821875 ,
         0.3821875 , 0.3821875 ],
        [0.40256435, 0.40256435, 0.40250257, ..., 0.40262613,
         0.40262613, 0.40256435],
        [0.42616642, 0.42616642, 0.42610195, ..., 0.42629647,
         0.42623153, 0.42616642],
        ...,
        [0.3389165 , 0.3389296 , 0.33895132, ..., 0.33904105,
         0.33899996, 0.33896253],
        [0.34599957, 0.34595275, 0.34596166, ..., 0.34603268,
         0.3460416 , 0.3459951 ],
        [0.3773438 , 0.3773438 , 0.3773438 , ..., 0.3773438 ,
         0.3773438 , 0.3773438 ]],

       [[0.38130265, 0.38130265, 0.38130265, ..., 0.38130265,
         0.38130265, 0.38130265],
        [0.4020675 , 0.4020675 , 0.40200636, ..., 0.40212923,
         0.40212923, 0.40212923],
        [0.42679802, 0.42679802, 0.42673355, ..., 0.42692837,
         0.42692837, 0.42686322],
        ...,
        [0.34001967, 0.3400371 , 0.34005457, ..., 0.34009007,
         0.3400526 , 0.3400109 ],
        [0.35121864, 0.3512277 , 0.35124126, ..., 0.3513696 ,
         0.35131758, 0.35132664],
        [0.3817628 , 0.3817628 , 0.3817628 , ..., 0.3817628 ,
         0.3817628 , 0.3817628 ]],

       [[0.38098764, 0.38098764, 0.38098764, ..., 0.38098764,
         0.38098764, 0.38098764],
        [0.40450966, 0.40438548, 0.40438548, ..., 0.40457124,
         0.40450966, 0.40450966],
        [0.43127406, 0.43120822, 0.43114245, ..., 0.43133983,
         0.43133983, 0.43127406],
        ...,
        [0.34072372, 0.3407918 , 0.3408049 , ..., 0.34084052,
         0.340803  , 0.34076115],
        [0.356262  , 0.35627577, 0.35628495, ..., 0.35629633,
         0.35630092, 0.35631007],
        [0.38455024, 0.38455024, 0.38455024, ..., 0.38455024,
         0.38455024, 0.38455024]],

       ...,

       [[0.39203107, 0.39203107, 0.39203107, ..., 0.39203107,
         0.39203107, 0.39203107],
        [0.41504985, 0.41504985, 0.41504985, ..., 0.4149863 ,
         0.4149863 , 0.4149863 ],
        [0.4289872 , 0.42892164, 0.42892164, ..., 0.4289872 ,
         0.4289872 , 0.4289872 ],
        ...,
        [0.46507218, 0.46501702, 0.4649737 , ..., 0.46494588,
         0.46496364, 0.46506035],
        [0.47772282, 0.47773504, 0.4776725 , ..., 0.47768617,
         0.47769836, 0.47770447],
        [0.4644957 , 0.4644957 , 0.4644957 , ..., 0.4644957 ,
         0.4644957 , 0.4644957 ]],

       [[0.38561893, 0.38561893, 0.38561893, ..., 0.38561893,
         0.38561893, 0.38561893],
        [0.40917006, 0.40923268, 0.40923268, ..., 0.40910375,
         0.40917006, 0.40917006],
        [0.4246037 , 0.4246037 , 0.42466858, ..., 0.42460003,
         0.42460003, 0.42466488],
        ...,
        [0.45510057, 0.4550465 , 0.45500377, ..., 0.4556914 ,
         0.45542297, 0.4551599 ],
        [0.4789382 , 0.47888228, 0.4788196 , ..., 0.47935697,
         0.47921985, 0.47915092],
        [0.46419963, 0.46419963, 0.46419963, ..., 0.46419963,
         0.46419963, 0.46419963]],

       [[0.37812507, 0.37812507, 0.37812507, ..., 0.37812507,
         0.37812507, 0.37812507],
        [0.40297312, 0.40297312, 0.4030348 , ..., 0.40284994,
         0.4029112 , 0.4029112 ],
        [0.41993132, 0.41993132, 0.41999504, ..., 0.4197993 ,
         0.41993132, 0.41993132],
        ...,
        [0.44608876, 0.44603556, 0.446064  , ..., 0.44659925,
         0.4463352 , 0.4461475 ],
        [0.48517463, 0.4851174 , 0.48512983, ..., 0.48544648,
         0.48530692, 0.48523742],
        [0.46785647, 0.46785647, 0.46785647, ..., 0.46785647,
         0.46785647, 0.46785647]]], dtype=float32)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.25 359.5 359.75
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 1980-01-01 ... 1980-12-31T23:00:00

In [10]:
dask_q = q.chunk({'latitude':100, 'longitude':100, 'time':500})/1000

In [11]:
dask_sp

,Array,Chunk
Bytes,36.48 GB,20.00 MB
Shape,"(8784, 721, 1440)","(500, 100, 100)"
Count,2161 Tasks,2160 Chunks
Type,float32,numpy.ndarray


In [12]:
%%time
tw = xr.apply_ufunc(WetBulb.WetBulb,
                        dask_mx2t, dask_sp, dask_q,
                        dask='parallelized',
#                         vectorize=True,
                        input_core_dims=None,
#                         output_core_dims=None,#[['latitude'], ['longitude'], ['time']],
#                         output_sizes={'latitude':dask_mx2t.latitude.size, 'longitude':dask_mx2t.longitude.size, 'time':dask_mx2t.time.size},
                        output_dtypes=[dask_q.dtype],
                        )

CPU times: user 4.01 ms, sys: 1.97 ms, total: 5.98 ms
Wall time: 5.1 ms


In [2]:
%%time

import warnings
warnings.filterwarnings('ignore')

# tw = tw.compute()
tw = tw.to_netcdf('%s/hourly/tw_%d.nc'%(dataDir, year))

In [23]:
tw.load()

<xarray.Dataset>
Dimensions:    (latitude: 721, longitude: 1440, time: 8784)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.25 359.5 359.75
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 1980-01-01 ... 1980-12-31T23:00:00
Data variables:
    tw         (time, latitude, longitude) float32 -25.426222 ... -27.541054